<h1><center>Problem Statements Solution </center></h1>

> **Problem Statement 1** -Using this source material, we want you to build a model that, based on a company website, can predict if the company is a venture capital firm or not.

   >  **Problem Statement 2** -Determine a list of keywords that can be used in a web search engine to discover new similar companies.

### Installing dependencies 

In [ ]:
%pip install wordfreq
%pip install pandas
%pip install sklearn
%pip install nltk
%pip install pandas
%pip install numpy

### Importing lib

In [1]:
import pandas as pd 
import numpy as np
import pickle
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn import model_selection
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score

## Importing required Lib 

In [2]:
df_vc = pd.read_pickle("Data_VC_Extracted_Scrape_1.pkl")
# Company = 1 means its is a VC
df_vc['Company'] = 1
print("Company = 1 means its is a VC")
df_vc.reset_index(drop=True, inplace=True)



df_non_vc =pd.read_pickle("Data_NON_VC_Extracted_Scrape_1.pkl")
# Company = 0 means NON_VC
df_non_vc['Company'] = 0
print("Company = 0 means NON_VC")
df_non_vc.reset_index(drop=True, inplace=True)

Company = 1 means its is a VC
Company = 0 means NON_VC


## Joining Both DataFrames

In [3]:
## Framing two columns as it might get out of shape
df_vc.columns = df_non_vc.columns
frame_combined = pd.concat([df_vc, df_non_vc], ignore_index=True)

In [4]:
data_df = frame_combined[['tags_in_home','Company']]
data_df['tags_in_home'] = data_df['tags_in_home'].astype(str)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Inititial pre processing - Removing na,chinease characters , numbers 

    1. Removing foreign language character

In [5]:
## Removing foreign language character
import string
printable = set(string.printable)
data_df['tags_in_home'] = data_df['tags_in_home'].apply(lambda row: ''.join(filter(lambda x: x in printable, row)))
data_df

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,tags_in_home,Company
0,Advent International Search Toggle About us Gl...,1
1,Skip to content Investors Investors Expertise ...,1
2,INVESTOR LOGIN About Investment Strategy Compe...,1
3,Home ABOUT Approach History INDUSTRY FOCUS POR...,1
4,"The J.E Robert Companies (""JER"") have ceased o...",1
5,About THL Our Culture & Values Our Investors a...,1
6,Skip to main content Portugus Franais English ...,1
7,Skip to main content MENU + About Overview Tea...,1
8,This site requires JavaScript and Cookies to b...,1
9,Investor Login visit our successor Home About ...,1


In [6]:
data_df['tags_in_home'].replace('', np.nan, inplace=True)

C:\Users\pc\Anaconda3\lib\site-packages\pandas\core\generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


    2. Removing na Rows

In [7]:
## Removing na Rows
data_df = data_df.dropna()
data_df.reset_index(inplace=True)
del data_df['index']
data_df

,tags_in_home,Company
0,Advent International Search Toggle About us Gl...,1
1,Skip to content Investors Investors Expertise ...,1
2,INVESTOR LOGIN About Investment Strategy Compe...,1
3,Home ABOUT Approach History INDUSTRY FOCUS POR...,1
4,"The J.E Robert Companies (""JER"") have ceased o...",1
5,About THL Our Culture & Values Our Investors a...,1
6,Skip to main content Portugus Franais English ...,1
7,Skip to main content MENU + About Overview Tea...,1
8,This site requires JavaScript and Cookies to b...,1
9,Investor Login visit our successor Home About ...,1


    3. Removing punctuation

In [8]:
def remove_punctuation(text):
    import string
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [9]:
data_df['new_text'] = data_df.apply(lambda x: remove_punctuation(x.tags_in_home), axis=1)
data_df

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,tags_in_home,Company,new_text
0,Advent International Search Toggle About us Gl...,1,Advent International Search Toggle About us Gl...
1,Skip to content Investors Investors Expertise ...,1,Skip to content Investors Investors Expertise ...
2,INVESTOR LOGIN About Investment Strategy Compe...,1,INVESTOR LOGIN About Investment Strategy Compe...
3,Home ABOUT Approach History INDUSTRY FOCUS POR...,1,Home ABOUT Approach History INDUSTRY FOCUS POR...
4,"The J.E Robert Companies (""JER"") have ceased o...",1,The JE Robert Companies JER have ceased ongoin...
5,About THL Our Culture & Values Our Investors a...,1,About THL Our Culture Values Our Investors an...
6,Skip to main content Portugus Franais English ...,1,Skip to main content Portugus Franais English ...
7,Skip to main content MENU + About Overview Tea...,1,Skip to main content MENU About Overview Team...
8,This site requires JavaScript and Cookies to b...,1,This site requires JavaScript and Cookies to b...
9,Investor Login visit our successor Home About ...,1,Investor Login visit our successor Home About ...


    4. Converting to lower_case

In [10]:
data_df['new_text'] = data_df['new_text'].str.lower()

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


    5. Removing numeric characters

In [11]:
## Removing number
data_df['new_text'] = data_df['new_text'].str.replace('\d+', '')

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
data_df

,tags_in_home,Company,new_text
0,Advent International Search Toggle About us Gl...,1,advent international search toggle about us gl...
1,Skip to content Investors Investors Expertise ...,1,skip to content investors investors expertise ...
2,INVESTOR LOGIN About Investment Strategy Compe...,1,investor login about investment strategy compe...
3,Home ABOUT Approach History INDUSTRY FOCUS POR...,1,home about approach history industry focus por...
4,"The J.E Robert Companies (""JER"") have ceased o...",1,the je robert companies jer have ceased ongoin...
5,About THL Our Culture & Values Our Investors a...,1,about thl our culture values our investors an...
6,Skip to main content Portugus Franais English ...,1,skip to main content portugus franais english ...
7,Skip to main content MENU + About Overview Tea...,1,skip to main content menu about overview team...
8,This site requires JavaScript and Cookies to b...,1,this site requires javascript and cookies to b...
9,Investor Login visit our successor Home About ...,1,investor login visit our successor home about ...


    7. Removing Stop Words

In [13]:
data_df

,tags_in_home,Company,new_text
0,Advent International Search Toggle About us Gl...,1,advent international search toggle about us gl...
1,Skip to content Investors Investors Expertise ...,1,skip to content investors investors expertise ...
2,INVESTOR LOGIN About Investment Strategy Compe...,1,investor login about investment strategy compe...
3,Home ABOUT Approach History INDUSTRY FOCUS POR...,1,home about approach history industry focus por...
4,"The J.E Robert Companies (""JER"") have ceased o...",1,the je robert companies jer have ceased ongoin...
5,About THL Our Culture & Values Our Investors a...,1,about thl our culture values our investors an...
6,Skip to main content Portugus Franais English ...,1,skip to main content portugus franais english ...
7,Skip to main content MENU + About Overview Tea...,1,skip to main content menu about overview team...
8,This site requires JavaScript and Cookies to b...,1,this site requires javascript and cookies to b...
9,Investor Login visit our successor Home About ...,1,investor login visit our successor home about ...


In [14]:
stop = stopwords.words('english')

In [15]:
def remove_stop(x):
    x = x.lower()
    stop = stopwords.words('english')
    x = [word for word in x.split() if x not in stop]

In [16]:
# data_df['unlisted_text'] = data_df.apply(lambda x: remove_stop(x.tags_in_home), axis=1)
# data_df

In [17]:
# data_df['new_text'] = [','.join(map(str, l)) for l in data_df['new_text']]

    6 . TokeniZing

In [18]:
tokenizer = RegexpTokenizer(r'\w+')

In [19]:
data_df

,tags_in_home,Company,new_text
0,Advent International Search Toggle About us Gl...,1,advent international search toggle about us gl...
1,Skip to content Investors Investors Expertise ...,1,skip to content investors investors expertise ...
2,INVESTOR LOGIN About Investment Strategy Compe...,1,investor login about investment strategy compe...
3,Home ABOUT Approach History INDUSTRY FOCUS POR...,1,home about approach history industry focus por...
4,"The J.E Robert Companies (""JER"") have ceased o...",1,the je robert companies jer have ceased ongoin...
5,About THL Our Culture & Values Our Investors a...,1,about thl our culture values our investors an...
6,Skip to main content Portugus Franais English ...,1,skip to main content portugus franais english ...
7,Skip to main content MENU + About Overview Tea...,1,skip to main content menu about overview team...
8,This site requires JavaScript and Cookies to b...,1,this site requires javascript and cookies to b...
9,Investor Login visit our successor Home About ...,1,investor login visit our successor home about ...


In [20]:
data_df['new_text'] = data_df['new_text'].apply(lambda x: tokenizer.tokenize(x.lower()))
# data_df

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


    8 . Lemmatizing

In [21]:

# # from nltk import PorterStemmer
# lemmatizer = WordNetLemmatizer()
# # stemmer = PorterStemmer()

# def word_lemmatizer(text):
#     lem_text = [lemmatizer.lemmatize(i) for i in text]
#     return lem_text

# # def word_stemmer(text):
# #     stem_text = [stemmer.stem(i) for i in text]
# #     return stem_text

In [22]:
# data_df['new_text'] = data_df['new_text'].apply(lambda x:word_lemmatizer(x))
# data_df['new_text'] = data_df['new_text'].apply(lambda x:word_stemmer(x))

## Removing Brackets

In [23]:
data_df

,tags_in_home,Company,new_text
0,Advent International Search Toggle About us Gl...,1,"[advent, international, search, toggle, about,..."
1,Skip to content Investors Investors Expertise ...,1,"[skip, to, content, investors, investors, expe..."
2,INVESTOR LOGIN About Investment Strategy Compe...,1,"[investor, login, about, investment, strategy,..."
3,Home ABOUT Approach History INDUSTRY FOCUS POR...,1,"[home, about, approach, history, industry, foc..."
4,"The J.E Robert Companies (""JER"") have ceased o...",1,"[the, je, robert, companies, jer, have, ceased..."
5,About THL Our Culture & Values Our Investors a...,1,"[about, thl, our, culture, values, our, invest..."
6,Skip to main content Portugus Franais English ...,1,"[skip, to, main, content, portugus, franais, e..."
7,Skip to main content MENU + About Overview Tea...,1,"[skip, to, main, content, menu, about, overvie..."
8,This site requires JavaScript and Cookies to b...,1,"[this, site, requires, javascript, and, cookie..."
9,Investor Login visit our successor Home About ...,1,"[investor, login, visit, our, successor, home,..."


In [24]:
data_df['unlisted_text'] = data_df['new_text'].agg(lambda x: ','.join(map(str, x)))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Diving Data 

In [25]:
 data_df[data_df['Company']==1]

,tags_in_home,Company,new_text,unlisted_text
0,Advent International Search Toggle About us Gl...,1,"[advent, international, search, toggle, about,...","advent,international,search,toggle,about,us,gl..."
1,Skip to content Investors Investors Expertise ...,1,"[skip, to, content, investors, investors, expe...","skip,to,content,investors,investors,expertise,..."
2,INVESTOR LOGIN About Investment Strategy Compe...,1,"[investor, login, about, investment, strategy,...","investor,login,about,investment,strategy,compe..."
3,Home ABOUT Approach History INDUSTRY FOCUS POR...,1,"[home, about, approach, history, industry, foc...","home,about,approach,history,industry,focus,por..."
4,"The J.E Robert Companies (""JER"") have ceased o...",1,"[the, je, robert, companies, jer, have, ceased...","the,je,robert,companies,jer,have,ceased,ongoin..."
5,About THL Our Culture & Values Our Investors a...,1,"[about, thl, our, culture, values, our, invest...","about,thl,our,culture,values,our,investors,and..."
6,Skip to main content Portugus Franais English ...,1,"[skip, to, main, content, portugus, franais, e...","skip,to,main,content,portugus,franais,english,..."
7,Skip to main content MENU + About Overview Tea...,1,"[skip, to, main, content, menu, about, overvie...","skip,to,main,content,menu,about,overview,team,..."
8,This site requires JavaScript and Cookies to b...,1,"[this, site, requires, javascript, and, cookie...","this,site,requires,javascript,and,cookies,to,b..."
9,Investor Login visit our successor Home About ...,1,"[investor, login, visit, our, successor, home,...","investor,login,visit,our,successor,home,about,..."


In [26]:
X = data_df['unlisted_text']
Y = data_df['Company']

## Problem 2 :  Determine a list of keywords that can be used in a web search engine to discover new similar companies (Assuming VC Site )(If required can be done for non-vc sites too)

In [27]:
corpus_vc = data_df[data_df['Company']==1]
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(corpus_vc)
corpus_vc = corpus_vc[['unlisted_text']]
corpus_vc

,unlisted_text
0,"advent,international,search,toggle,about,us,gl..."
1,"skip,to,content,investors,investors,expertise,..."
2,"investor,login,about,investment,strategy,compe..."
3,"home,about,approach,history,industry,focus,por..."
4,"the,je,robert,companies,jer,have,ceased,ongoin..."
5,"about,thl,our,culture,values,our,investors,and..."
6,"skip,to,main,content,portugus,franais,english,..."
7,"skip,to,main,content,menu,about,overview,team,..."
8,"this,site,requires,javascript,and,cookies,to,b..."
9,"investor,login,visit,our,successor,home,about,..."


In [28]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(corpus_vc['unlisted_text'])
# corpus_tfidf = Tfidf_vect.transform(corpus_vc)
# Test_X_Tfidf = Tfidf_vect.transform(X_test)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

# *Problem 2* -  Key Words
### List of key words for VC companies . Similiarly if required for NON VC companies we can fid out important keywords which can be used for text search for finding similar sites

In [29]:
print(Tfidf_vect.vocabulary_)

{'advent': 98, 'international': 2204, 'search': 3694, 'toggle': 4199, 'about': 4, 'us': 4389, 'global': 1803, 'reach': 3423, 'sectors': 3700, 'investments': 2232, 'team': 4105, 'news': 2835, 'contact': 932, 'overview': 2997, 'investment': 2231, 'strategy': 3975, 'how': 2023, 'we': 4517, 'add': 65, 'value': 4412, 'community': 845, 'diversity': 1202, 'and': 209, 'inclusion': 2107, 'asia': 293, 'greater': 1846, 'china': 730, 'india': 2117, 'europe': 1467, 'france': 1710, 'germany': 1786, 'luxembourg': 2528, 'spain': 3868, 'united': 4356, 'kingdom': 2352, 'latin': 2403, 'america': 190, 'talent': 4085, 'programs': 3310, 'in': 2097, 'latam': 2396, 'brazil': 526, 'colombia': 818, 'mexico': 2682, 'peru': 3119, 'north': 2862, 'states': 3938, 'business': 577, 'financial': 1605, 'services': 3744, 'healthcare': 1927, 'industrial': 2123, 'retail': 3555, 'consumer': 930, 'leisure': 2431, 'technology': 4113, 'case': 656, 'studies': 3999, 'ipos': 2246, 'welcome': 4531, 'is': 2254, 'one': 2925, 'of': 2

# *Problem 1* -  Key Words
### Classifier that can predict if a website belongs to VC or not 

### Count Vectorizing 

In [30]:
X

0      advent,international,search,toggle,about,us,gl...
1      skip,to,content,investors,investors,expertise,...
2      investor,login,about,investment,strategy,compe...
3      home,about,approach,history,industry,focus,por...
4      the,je,robert,companies,jer,have,ceased,ongoin...
5      about,thl,our,culture,values,our,investors,and...
6      skip,to,main,content,portugus,franais,english,...
7      skip,to,main,content,menu,about,overview,team,...
8      this,site,requires,javascript,and,cookies,to,b...
9      investor,login,visit,our,successor,home,about,...
10     horsley,bridge,overview,people,contact,home,ov...
11     created,using,figma,home,about,team,portfolio,...
12     home,firm,investment,support,group,environment...
13     home,firm,overview,investment,criteria,the,tea...
14     bedford,is,an,excellent,business,partner,for,m...
15     the,yucaipa,companies,home,contact,managing,pa...
16     we,see,venture,capital,the,way,you,see,your,bu...
17     about,us,portfolio,team,

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words=stopwords.words('english'))
count_vec = vectorizer.fit_transform(X).toarray()
# X = vectorizer.fit_transform(X_dat).toarray()

## Splitting into Train & Test & Validation Set's

In [32]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X,Y, test_size=0.25, random_state=0)

In [33]:
Y_test.shape

(39,)

In [34]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data_df['unlisted_text'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [35]:
# print(Train_X_Tfidf)

## Naive Bayes

In [36]:
# fit the training dataset on the NB classifier
from sklearn.naive_bayes import MultinomialNB
Naive = MultinomialNB()
Naive.fit(Train_X_Tfidf,Y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Y_test)*100)

Naive Bayes Accuracy Score ->  48.717948717948715


In [37]:
print('Accuracy score: ', accuracy_score(Y_test, predictions_NB))
print('Precision score: ', precision_score(Y_test, predictions_NB))
print('Recall score: ', recall_score(Y_test, predictions_NB))

Accuracy score:  0.48717948717948717
Precision score:  0.4594594594594595
Recall score:  1.0


## SVM

In [38]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data_df['unlisted_text'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [39]:
from sklearn import svm
naive_bayes = svm.SVC()
naive_bayes.fit(Train_X_Tfidf, Y_train)
predictions_svm = naive_bayes.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_svm, Y_test)*100)

SVM Accuracy Score ->  43.58974358974359


C:\Users\pc\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [40]:
print('Accuracy score: ', accuracy_score(Y_test, predictions_svm))
print('Precision score: ', precision_score(Y_test, predictions_svm))
print('Recall score: ', recall_score(Y_test, predictions_svm))

Accuracy score:  0.4358974358974359
Precision score:  0.4358974358974359
Recall score:  1.0


## Using CountVectoriser

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [47]:
df = data_df[['unlisted_text','Company']].copy()
counter = df['Company'].tolist()
# variety_list = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
# df = df[df['Company'].map(lambda x: x in top_10_varieties)]

tas_list = df['unlisted_text'].tolist()
\
varietal_list = np.array(df['Company'])

count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(tas_list)


tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, varietal_list, test_size=0.3)

clf = MultinomialNB().fit(train_x, train_y)
y_score = clf.predict(test_x)

n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1



Accuracy: 70.21%


In [48]:
print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 70.21%


### End